In [43]:
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import requests
from bs4 import BeautifulSoup
import json

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [45]:

#URL = 'https://www.rbc.ru/neweconomy/'
URL = 'https://www.buhgalteria.ru'


def save_json(data):
    with open('buh_data.json', "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

def get_soup(url:str, params:dict=dict()):
    r = requests.get(url, params)
    r.encoding = r.apparent_encoding
    if r.status_code != 200:
        raise Exception()
    return BeautifulSoup(r.text, "lxml")

def get_page_links(rubric_url):
    soup = get_soup(rubric_url)
    return [{"rubric": rubric_url.split("/")[-2], "href": j["href"], "title": j.text}
            for j in [i.find("a") for i in soup.find_all("h3")]
            if "article" in j["href"]]

def get_all_article_links():
    main_page_url = "https://www.buhgalteria.ru/"
    soup = get_soup(main_page_url)
    categories = [i["href"] for i in soup.find("ul", {"class": "dropnav-menu"}).find_all("a")]
    categories = [href for href in categories if "rubric" in href]

    all_page_links = list()
    all_page_hrefs = set()
    for rubric in categories:
        page_links = get_page_links(rubric_url=rubric)
        for link in page_links:
            link["href"] = link["href"].split("?")[0]
            if "https://www.buhgalteria.ru" not in link["href"]:
                link["href"] = "https://www.buhgalteria.ru" + link["href"]
            if link["href"] not in all_page_hrefs:
                all_page_hrefs.add(link["href"])
                all_page_links.append(link)
    return all_page_links

all_pages = get_all_article_links()
#print(all_pages)
def main():
    news_data = {}

    soup = get_soup(URL)
    news_links = []
    for i in all_pages:
        news_links.append(i['href'])
    # Для каждой ссылки получаем информацию и записываем в news_data
    for i in range(len(news_links)):

        #link = news_links[i].get('href').split('?')[0]
        link = news_links[i]
        name = link
        news_data[name] = {}
        soup = get_soup(link)

        # Переходим на страницу для дальнейшенго парсинга
        article = soup.find('div', class_='article')
        #print(article.find('a'))
        category = article.find('a', class_='article__header__category')

        #date = article.find('span', class_='article__header__date').get('content').replace('T', ' ').split('+')[0]

        title = article.find('h1')
        image = article.find('div', class_='article__main-image')

        article_paragraphs = article.find_all('p')
        article_text = ''
        for paragraph in article_paragraphs:
            article_text += paragraph.text

        # Заполняем полученными данными news_data
        news_data[name]['link'] = link
        #news_data[name]['date'] = date
        news_data[name]['text'] = article_text.replace('\xa0', '').replace('\n', '').replace('\r', '')

        try:
            news_data[name]['title'] = title.text.strip()
        except AttributeError:
            news_data[name]['title'] = 'Без заголовка'
        try:
            news_data[name]['category'] = category.text.replace('\n', '')
        except AttributeError:
            news_data[name]['category'] = 'B'

    save_json(news_data)
    frame = pd.DataFrame(news_data).transpose()

    frame.to_csv("buh_data.csv", )



if __name__ == "__main__":
    main()

In [ ]:
save_json(news_data)
frame = pd.DataFrame(news_data).transpose()

frame.to_csv("buh_data.csv", )